In [1]:
"""
# 내장 자료형의 상속은 까다로움

- CPython의 경우, 오버라이드한 메서드에 대해 호출과 관련해서 명확한 규칙을 정의하지 않음
- 예를들어, dict의 서브클래스에서 오버라이드한 __getitem__() 메서드는 내장된 get()과 같은 메서드에 의해 호출안됨

# 내장 자료형은 예외 케이스

- 슈퍼 클래스에서 구현된 메서드 안에서 호출하더라도 메서드 검색은 대상 객체(self)의 클래스에서 시작해야함을 어김
"""

class DoppelDict(dict) : 
    def __setitem__(self, key, value) :
        super().__setitem__(key, [value] * 2)
        
dd = DoppelDict(one=1)
dd

{'one': 1}

In [2]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [4]:
dd.update(three=3) # update() 에서 오버라이드된 __setitem__() 메서드를 호출하지 않음 
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [5]:
class AnswerDict(dict) : 
    def __getitem__(self, key) : 
        return 42
    
ad = AnswerDict(a='foo')
ad['a']

42

In [6]:
d = {}
d.update(ad)
d['a']

'foo'

In [13]:
"""
위의 문제를 collections.UserDict으로 해결 가능함
"""
import collections

class DoppelDict2(collections.UserDict) : 
    def __setitem__(self, key, value) : 
        super().__setitem__(key, [value] * 2)
        
class AnswerDict2(collections.UserDict) : 
    def __getitem__(self, key) : 
        return 42
        
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [14]:
dd['two'] = 2

In [15]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [16]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [17]:
d = {}
d.update(ad)
d['a']

42

In [27]:
"""
# 다중 상속과 메서드 결정 순서

- 이름 출동 문제 발생
- 밑에 예제는 다이아몬드 문제
"""

class A : 
    def ping(self) : 
        print('ping : ', self)
        
class B(A) : 
    def pong(self) : 
        print('pong : ', self)
        
class C(A) : 
    def pong(self) : 
        print('PONG : ', self)
        
class D(B, C) : 
    def ping(self) : 
        super().ping()
        print('post-ping : ', self)
        
    def pingpong(self) : 
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)


In [28]:
d = D()
d.pong()

pong :  <__main__.D object at 0x7fa010991be0>


In [29]:
C.pong(d)

PONG :  <__main__.D object at 0x7fa010991be0>


In [30]:
"""
- 파이썬이 상속 그래프를 조회할 때는 특정한 순서를 따르므로, d.pong()과 같은 호출의 모호함이 해결됨
- 이때, MRO(메서드 결정 순서)를 따름 
- MRO는 상속 그래프를 고려하고 서브클래스 정의에 나열된 슈퍼클래스들의 순서도 고려함 
"""

D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [31]:
d.ping()

ping :  <__main__.D object at 0x7fa010991be0>
post-ping :  <__main__.D object at 0x7fa010991be0>


In [32]:
d.pingpong()

ping :  <__main__.D object at 0x7fa010991be0>
post-ping :  <__main__.D object at 0x7fa010991be0>
ping :  <__main__.D object at 0x7fa010991be0>
pong :  <__main__.D object at 0x7fa010991be0>
pong :  <__main__.D object at 0x7fa010991be0>
PONG :  <__main__.D object at 0x7fa010991be0>


In [35]:
"""
__mro__를 통해서 클래스 계층 구조 파악할 수 있음
"""

bool.__mro__

(bool, int, object)

In [39]:
"""
Tkinter의 다중 상속 그래프 확인
"""

import tkinter

def print_mro(cls) : 
    print(', '.join(c.__name__ for c in cls.__mro__))
    
print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object
